# Install Required Libraries

In [1]:
!pip install yt-dlp
!pip install pydrive
!pip install pydub
!pip install openai
!pip install whisper

UndefVarError: UndefVarError: `pip` not defined

# Import Libraries and Define Functions

In [2]:
import yt_dlp
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydub import AudioSegment
import openai
import whisper
import os
import shutil
from IPython.display import display, HTML
import ipywidgets as widgets

# Authenticate Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

# Create directories
os.makedirs('/tmp/downloads', exist_ok=True)

def download_youtube_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': '/tmp/downloads/%(title)s.%(ext)s'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def download_drive_video(file_id):
    file = drive.CreateFile({'id': file_id})
    file.GetContentFile('/tmp/downloads/video.mp4')
    
    audio = AudioSegment.from_file('/tmp/downloads/video.mp4')
    audio.export('/tmp/downloads/audio.mp3', format='mp3')

def generate_transcript(file_path, api_key):
    openai.api_key = api_key
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result['text']

def generate_captions(transcript):
    lines = transcript.split('\n')
    captions = []
    for line in lines:
        if ']' in line:
            timestamp, text = line.split('] ', 1)
            captions.append(f"{timestamp}] {text}")
    return "\n".join(captions)

def clean_up():
    if os.path.exists('/tmp/downloads'):
        shutil.rmtree('/tmp/downloads')

ArgumentError: ArgumentError: Package yt_dlp not found in current path.
- Run `import Pkg; Pkg.add("yt_dlp")` to install the yt_dlp package.

# Create UI Elements

In [3]:
# UI elements
api_key_input = widgets.Text(
    value='',
    placeholder='Enter your OpenAI API key',
    description='API Key:',
    disabled=False
)

url_input = widgets.Text(
    value='',
    placeholder='Enter the video URL',
    description='Video URL:',
    disabled=False
)

file_type_input = widgets.Dropdown(
    options=['youtube', 'drive'],
    value='youtube',
    description='File Type:',
    disabled=False,
)

keywords_input = widgets.Text(
    value='',
    placeholder='Enter keywords, separated by commas',
    description='Keywords:',
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',
    tooltip='Submit the form',
    icon='check'
)

output = widgets.Output()

def on_submit(b):
    with output:
        output.clear_output()
        api_key = api_key_input.value
        url = url_input.value
        file_type = file_type_input.value
        keywords = keywords_input.value
        
        # Validate inputs
        if not api_key or not url or not keywords:
            print("Please fill in all fields.")
            return
        
        # Process video
        try:
            os.makedirs('/tmp/downloads', exist_ok=True)
            if file_type == 'youtube':
                download_youtube_audio(url)
                file_path = '/tmp/downloads/audio.mp3'
            elif file_type == 'drive':
                file_id = url.split('id=')[1]
                download_drive_video(file_id)
                file_path = '/tmp/downloads/audio.mp3'

            transcript = generate_transcript(file_path, api_key)
            captions = generate_captions(transcript)

            # Display results
            display(HTML(f"<h3>Transcript:</h3><pre>{transcript}</pre>"))
            display(HTML(f"<h3>Captions:</h3><pre>{captions}</pre>"))
            display(HTML(f"<h3>Keywords:</h3><pre>{keywords}</pre>"))
        except Exception as e:
            print(f"Error processing video: {e}")
        finally:
            clean_up()

submit_button.on_click(on_submit)

# Display UI
display(api_key_input, url_input, file_type_input, keywords_input, submit_button, output)

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\Employee\OneDrive\Desktop\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:3:12
api_key_input = widgets.Text(
    value='',
#          └ ── empty character literal